# Use `ocetrac-unstruct` to identify and track marine heatwaves
This example using 40 years of Daily outputs at 5km native grid resolution takes ~ minutes on 128 cores.

In [ ]:
import xarray as xr
import numpy as np
import dask
import ocetrac_unstruct
import pyicon as pyic  # Necessary for unstructured plotting of ICON data

from tempfile import TemporaryDirectory
from getpass import getuser
from pathlib import Path
from dask.distributed import Client, LocalCluster
import subprocess
import re

import warnings
warnings.filterwarnings('ignore')

In [2]:
scratch_dir = Path('/scratch') / getuser()[0] / getuser() / 'mhws' 
zarr_fpath_tgrid = scratch_dir  / f'fpath_tgrid.zarr'

## Start Dask Cluster

In [ ]:
cluster_scratch = Path('/scratch') / getuser()[0] / getuser() / 'clients'
dask_tmp_dir = TemporaryDirectory(dir=cluster_scratch)
dask.config.set(temporary_directory=dask_tmp_dir.name)

## Local Cluster
cluster = LocalCluster(n_workers=16, threads_per_worker=4)
client = Client(cluster)

remote_node = subprocess.run(['hostname'], capture_output=True, text=True).stdout.strip().split('.')[0]
port = re.search(r':(\d+)/', client.dashboard_link).group(1)
print(f"Forward with Port = {remote_node}:{port}")

client.dashboard_link

2024-09-29 00:29:08,661 - distributed.nanny - WARNING - Restarting worker
2024-09-29 00:29:08,669 - distributed.nanny - WARNING - Restarting worker
2024-09-29 00:29:08,684 - distributed.nanny - WARNING - Restarting worker
2024-09-29 00:29:08,686 - distributed.nanny - WARNING - Restarting worker
2024-09-29 00:29:08,689 - distributed.nanny - WARNING - Restarting worker
2024-09-29 00:29:08,850 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:33067' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('getitem-df310c0aebeab914d9c8c5f84782cd40', 47), ('getitem-df310c0aebeab914d9c8c5f84782cd40', 44), ('getitem-62839115bb8c676c9c7a96325c920b0d', 33, 0), ('getitem-df310c0aebeab914d9c8c5f84782cd40', 4), ('getitem-df310c0aebeab914d9c8c5f84782cd40', 1), ('getitem-df310c0aebeab914d9c8c5f84782cd40', 7), ('getitem-df310c0aebeab914d9c8c5f84782cd40', 10), ('getitem-df310c0aebeab914d9c8c5f84782cd40', 16), ('getitem-df310c0aebeab914d9c8c5f84782cd

## Load Pre-processed Data
(cf. `01_preprocess_unstruct.ipynb`)

In [4]:
chunk_size = {'time': 4, 'ncells': -1}
time_subset = slice(0,200)
ds = xr.open_zarr(str(scratch_dir / '01_preprocess_unstruct.zarr'), chunks=chunk_size).isel(time=time_subset)

In [ ]:
binary_out = ds.features_notrend.persist()
binary_out

In [6]:
mask = ds.mask.compute()

In [7]:
# binary_out.isel(time=0).pyic.plot(fpath_tgrid=zarr_fpath_tgrid)

## Run Tracker

In [8]:
# Tracking Parameters
threshold_percentile = 0.95
min_size_quartile = 0.5       # percent
radius = 100.0                # km
resolution = 5.0              # km

In [9]:
tracker = ocetrac_unstruct.Tracker(binary_out, radius=radius, resolution=resolution, min_size_quartile=min_size_quartile, timedim='time', xdim='ncells', neighbours=ds.neighbours, land_mask=mask)

In [10]:
binary_images = tracker._morphological_operations()

In [11]:
#binary_images = binary_images.persist()

In [ ]:
area, min_area, binary_images_filtered, N_initial = tracker._filter_area(binary_images)

In [ ]:
## ^^^ WORKS 

In [ ]:
labelled_global_time, N_final = tracker._label_unstruct_time(binary_images_filtered)

In [ ]:
blobs = tracker.track()

In [10]:
blobs.attrs

## Save Blobs

In [11]:
blobs.to_netcdf(scratch_dir / '02_tracked_unstruct.nc', mode='w')